In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import streamlit as st

In [2]:
# Loading the dataset
df = pd.read_csv(r'C:\Users\USER\Downloads\Expresso_churn_dataset.csv')
# Display general information about the dataset
print(df.info())
# Display general information about the dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154048 entries, 0 to 2154047
Data columns (total 19 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         object 
 1   REGION          object 
 2   TENURE          object 
 3   MONTANT         float64
 4   FREQUENCE_RECH  float64
 5   REVENUE         float64
 6   ARPU_SEGMENT    float64
 7   FREQUENCE       float64
 8   DATA_VOLUME     float64
 9   ON_NET          float64
 10  ORANGE          float64
 11  TIGO            float64
 12  ZONE1           float64
 13  ZONE2           float64
 14  MRG             object 
 15  REGULARITY      int64  
 16  TOP_PACK        object 
 17  FREQ_TOP_PACK   float64
 18  CHURN           int64  
dtypes: float64(12), int64(2), object(5)
memory usage: 312.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154048 entries, 0 to 2154047
Data columns (total 19 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         object 
 1   REG

In [3]:
# Display summary statistics
print(df.describe())


            MONTANT  FREQUENCE_RECH       REVENUE  ARPU_SEGMENT     FREQUENCE  \
count  1.397309e+06    1.397309e+06  1.428000e+06  1.428000e+06  1.428000e+06   
mean   5.532117e+03    1.152912e+01  5.510810e+03  1.836943e+03  1.397814e+01   
std    7.111339e+03    1.327407e+01  7.187113e+03  2.395700e+03  1.469403e+01   
min    1.000000e+01    1.000000e+00  1.000000e+00  0.000000e+00  1.000000e+00   
25%    1.000000e+03    2.000000e+00  1.000000e+03  3.330000e+02  3.000000e+00   
50%    3.000000e+03    7.000000e+00  3.000000e+03  1.000000e+03  9.000000e+00   
75%    7.350000e+03    1.600000e+01  7.368000e+03  2.456000e+03  2.000000e+01   
max    4.700000e+05    1.330000e+02  5.321770e+05  1.773920e+05  9.100000e+01   

        DATA_VOLUME        ON_NET        ORANGE           TIGO          ZONE1  \
count  1.093615e+06  1.367373e+06  1.258800e+06  864032.000000  169721.000000   
mean   3.366450e+03  2.776891e+02  9.541871e+01      23.109253       8.170132   
std    1.330446e+04  8.7268

In [4]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Expresso Churn Profiling Report", explorative=True)
profile.to_file("expresso_churn_report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Check for missing values
print(df.isnull().sum())


user_id                 0
REGION             849299
TENURE                  0
MONTANT            756739
FREQUENCE_RECH     756739
REVENUE            726048
ARPU_SEGMENT       726048
FREQUENCE          726048
DATA_VOLUME       1060433
ON_NET             786675
ORANGE             895248
TIGO              1290016
ZONE1             1984327
ZONE2             2017224
MRG                     0
REGULARITY              0
TOP_PACK           902594
FREQ_TOP_PACK      902594
CHURN                   0
dtype: int64


In [6]:
# Handle missing values (example: filling with median for numerical features)
for col in df.select_dtypes(include=[np.number]).columns:
    df[col].fillna(df[col].median(), inplace=True)

# For categorical features, filling missing values with the mode
for col in df.select_dtypes(include=[object]).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

    # Check for duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove duplicates if they exist
df.drop_duplicates(inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_10648\3298092293.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_10648\3298092293.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

Number of duplicate rows: 0


In [7]:
# Identify and handle outliers using IQR method
for col in df.select_dtypes(include=[np.number]).columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))]
    if not outliers.empty:
        print(f"Handling outliers in column: {col}")
# For simplicity, let's clip the outliers to the 1.5*IQR range
        df[col] = np.clip(df[col], Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)


Handling outliers in column: MONTANT
Handling outliers in column: FREQUENCE_RECH
Handling outliers in column: REVENUE
Handling outliers in column: ARPU_SEGMENT
Handling outliers in column: FREQUENCE
Handling outliers in column: DATA_VOLUME
Handling outliers in column: ON_NET
Handling outliers in column: ORANGE
Handling outliers in column: TIGO
Handling outliers in column: ZONE1
Handling outliers in column: ZONE2
Handling outliers in column: FREQ_TOP_PACK
Handling outliers in column: CHURN


In [8]:
# Reduce cardinality for high cardinality categorical features
def reduce_cardinality(series, threshold=0.01):
    freq = series.value_counts(normalize=True)
    other_categories = freq[freq < threshold].index
    return series.apply(lambda x: 'Other' if x in other_categories else x)


In [9]:
# Select features and target
selected_features = ['REVENUE', 'FREQUENCE_RECH', 'MONTANT', 'CHURN']
X = df[selected_features]
y = df['CHURN']


In [10]:
# Encode categorical features using sparse data structures
categorical_features = df.select_dtypes(include=['object']).columns
df_sparse = pd.get_dummies(df, columns=categorical_features, drop_first=True, sparse=True)


In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=False) 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [13]:
# Initialize and train the classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [14]:
# Make predictions
y_pred = clf.predict(X_test)


In [15]:
# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))


Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    430810

    accuracy                           1.00    430810
   macro avg       1.00      1.00      1.00    430810
weighted avg       1.00      1.00      1.00    430810

Confusion Matrix:
[[430810]]


c:\Users\USER\anaconda3_2\Lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [16]:
# Saving the model
import pickle
filename = "trained_model.sav"
pickle.dump(RandomForestClassifier, open(filename, "wb"))